# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

Import the dataset with mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json

In [1]:
# Imports dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Creates instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# Confirms database creation
pprint(mongo.list_database_names())

['admin',
 'airline_data',
 'classDB',
 'config',
 'fruits_db',
 'gardenDB',
 'local',
 'met',
 'petsitly_marketing',
 'uk_food']


In [4]:
# Assigns uk_food database to a variable name
food = mongo['uk_food']

In [5]:
# Reviews collections in our new database
pprint(food.list_collection_names())

['establishments']


In [6]:
# Reviews document in establishments collection
pprint(food.establishments.find_one())

{'AddressLine1': '27 Granville Road',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Lenox House',
 'BusinessType': 'Hotel/bed & breakfast/guest house',
 'BusinessTypeID': 7842,
 'ChangesByServerID': 0,
 'Distance': 4587.440906956158,
 'FHRSID': 632212,
 'LocalAuthorityBusinessID': 'PI/000043474',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DS',
 'RatingDate': '2020-03-17T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('641b91f37d831d3ccbd58f3e'),
 'geocode': {'latitude': '51.152411', 'longitude': '1.385585'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/632212',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDat

In [6]:
# Assigns collection to variable
establishments = food['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [7]:
# Creates dictionary for new restaurant data
penang_flavours = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [8]:
# Inserts new restaurant into collection
establishments.insert_one(penang_flavours)

In [9]:
# Checks that new restaurant was inserted
query = {'BusinessName':'Penang Flavours'}
result = establishments.find(query)

pprint([item for item in result])

[{'AddressLine1': 'Penang Flavours',
  'AddressLine2': '146A Plumstead Rd',
  'AddressLine3': 'London',
  'AddressLine4': '',
  'BusinessName': 'Penang Flavours',
  'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  'Distance': 4623.972328074718,
  'LocalAuthorityCode': '511',
  'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
  'LocalAuthorityName': 'Greenwich',
  'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
  'NewRatingPending': True,
  'Phone': '',
  'PostCode': 'SE18 7DY',
  'RightToReply': '',
  'SchemeType': 'FHRS',
  '_id': ObjectId('641b92718b7b1f786bf4d5ad'),
  'geocode': {'latitude': 51.490142, 'longitude': 0.08384},
  'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}},
 {'AddressLine1': 'Penang Flavours',
  'AddressLine2': '146A Plumstead Rd',
  'AddressLine3': 'London',
  'AddressLine4': '',
  'BusinessName': 'Penang Flavours',
  'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': '',
  'Distan

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [12]:
# Finds BusinessTypeID for "Restaurant/Cafe/Canteen" and returns only BusinessTypeID and BusinessType fields
query = {'BusinessType': 'Restaurant/Cafe/Canteen'}
fields = {'_id': 0, 'BusinessTypeID': 1, 'BusinessType': 1}
result = establishments.find(query, fields)

pprint([result[i] for i in range(1)])

[{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}]


3. Update the new restaurant with the `BusinessTypeID` you found.

In [13]:
# Updates new restaurant with the correct BusinessTypeID
query = {'BusinessName': 'Penang Flavours'}
setter = {'$set': {"BusinessTypeID": 1}}
establishments.update_one(query, setter)

In [14]:
# Confirms that new restaurant was updated
query = {'BusinessName': 'Penang Flavours'}
fields = {'_id': 0, 'BusinessName': 1, 'BusinessTypeID': 1}
result = establishments.find(query, fields)
pprint(result[0])

{'BusinessName': 'Penang Flavours', 'BusinessTypeID': 1}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [16]:
# Finds how many documents have LocalAuthorityName "Dover"
query = {'LocalAuthorityName': 'Dover'}
print(establishments.count_documents(query))

0


In [17]:
# Deletes all documents where LocalAuthorityName is "Dover"
establishments.delete_many(query)

In [18]:
# Check any remaining documents including Dover
query = {'LocalAuthorityName': 'Dover'}
print(establishments.count_documents(query))

0


In [19]:
# Checks that other documents remain with 'find_one'
establishments.find_one()

{'_id': ObjectId('641b91f37d831d3ccbd59225'),
 'FHRSID': 1043695,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'PostCode': 'CT19 6BL',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2018-04-04T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': 1.195625, 'latitude': 51.083812},
 'RightToReply': '',
 'Distance': 4591.765489457773,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:

5. Some of the number values are stored as strings, when they should be stored as numbers. Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [20]:
# Changes data type from String to Decimal for longitude 
establishments.update_many({},[{'$set':{"geocode.longitude":{'$toDouble': "$geocode.longitude"}}}])

In [21]:
# Changes data type from String to Decimal for latitude
establishments.update_many({},[{'$set':{"geocode.latitude":{'$toDouble': "$geocode.latitude"}}}])

In [20]:
# Checks that coordinates now numbers
establishments.find_one()

{'_id': ObjectId('641b91f37d831d3ccbd59225'),
 'FHRSID': 1043695,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'PostCode': 'CT19 6BL',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2018-04-04T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': 1.195625, 'latitude': 51.083812},
 'RightToReply': '',
 'Distance': 4591.765489457773,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00: